### 2 数据加载
为了训练匹配模型，一般需要准备三个数据集：训练集 train.tsv、验证集dev.tsv、测试集test.tsv。此案例我们使用 PaddleNLP 内置的语义数据集 [LCQMC](http://icrc.hitsz.edu.cn/Article/show/171.html) 来进行训练、评估、预测。

训练集: 用来训练模型参数的数据集，模型直接根据训练集来调整自身参数以获得更好的分类效果。

验证集: 用于在训练过程中检验模型的状态，收敛情况。验证集通常用于调整超参数，根据几组模型验证集上的表现，决定采用哪组超参数。

测试集: 用来计算模型的各项评估指标，验证模型泛化能力。

[LCQMC](http://icrc.hitsz.edu.cn/Article/show/171.html) 数据集是公开的语义匹配权威数据集。PaddleNLP 已经内置该数据集，一键即可加载。

In [1]:
# !pip install -U paddlepaddle -i https://mirror.baidu.com/pypi/simple

# 正式开始实验之前首先通过如下命令安装最新版本的 paddlenlp
# !pip install --upgrade paddlenlp -i https://pypi.org/simple

!pip install paddle-ernie
!rm -Rf checkpoint/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import time
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm

import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddlenlp.datasets import load_dataset
import paddlenlp
from paddlenlp.transformers import BertForSequenceClassification, BertTokenizer
from ernie.tokenizing_ernie import ErnieTokenizer

In [3]:
EPOCHS = 1
# EPOCHS = 5
# EPOCHS = 20
# EPOCHS = 50

# BATCH_SIZE = 14
# BATCH_SIZE = 32
BATCH_SIZE = 40
# BATCH_SIZE = 50
# BATCH_SIZE = 64

# LR = 5E-5
LR = 1E-5

MAX_SEQ_LENGTH = 512
# MAX_SEQ_LENGTH = 1024

# EVAL = True
EVAL = False
TEST_SIZE = 0.1

# EVAL_STEP = 100
# EVAL_STEP = 500
# EVAL_STEP = 1000
EVAL_STEP = 5000

# TRAIN_CSV = 'org619885.csv'
# TRAIN_CSV = 'bug618504.csv'
# TRAIN_CSV = 'crop618504.csv'
TRAIN_CSV = 'chs_619885.csv'

# TEST_CSV = 'test_public.csv'
TEST_CSV = 'chs_test_public.csv'

tokenizer = paddlenlp.transformers.ErnieGramTokenizer.from_pretrained('ernie-gram-zh')
pretrained_model = paddlenlp.transformers.ErnieGramModel.from_pretrained('ernie-gram-zh')
# tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained('ernie-1.0')
# pretrained_model = paddlenlp.transformers.ErnieModel.from_pretrained('ernie-1.0')
# tokenizer = paddlenlp.transformers.BertTokenizer.from_pretrained('bert-base-chinese')
# pretrained_model = paddlenlp.transformers.BertModel.from_pretrained('bert-base-chinese')
# tokenizer = paddlenlp.transformers.AlbertTokenizer.from_pretrained('albert-chinese-base')
# pretrained_model = paddlenlp.transformers.AlbertModel.from_pretrained('albert-chinese-base')

[2021-11-24 00:34:11,951] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-gram-zh/vocab.txt
[2021-11-24 00:34:11,965] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-gram-zh/ernie_gram_zh.pdparams
W1124 00:34:11.967811   440 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1124 00:34:11.972581   440 device_context.cc:465] device: 0, cuDNN Version: 7.6.


In [4]:
# 一键加载 Lcqmc 的训练集、验证集
# train_ds, dev_ds = load_dataset("lcqmc", splits=["train", "dev"])


def read(data_path):
    for idx, row in data_path.iterrows():
        yield {'query': row.Test_text, 'title': row.Reference_text, 'label': row.sentiment}


train_df = pd.read_csv(f'data/data117130/{TRAIN_CSV}')

if EVAL:
    train_df, val_df = train_test_split(train_df, test_size=TEST_SIZE, random_state=777,  shuffle=True)

# data_path为read()方法的参数
train_ds = load_dataset(read, data_path=train_df, lazy=False)

if EVAL:
    dev_ds = load_dataset(read, data_path=val_df, lazy=False)
    print(len(train_ds), len(dev_ds))
else:
    print(len(train_ds))

619885


### 2.2 数据预处理

通过 PaddleNLP 加载进来的 [LCQMC](http://icrc.hitsz.edu.cn/Article/show/171.html) 数据集是原始的明文数据集，这部分我们来实现组 batch、tokenize 等预处理逻辑，将原始明文数据转换成网络训练的输入数据。

#### 定义样本转换函数

In [5]:
# 将 1 条明文数据的 query、title 拼接起来，根据预训练模型的 tokenizer 将明文转换为 ID 数据
# 返回 input_ids 和 token_type_ids

def convert_example(example, tokenizer, max_seq_length=512, is_test=False):

    query, title = example["query"], example["title"]

    encoded_inputs = tokenizer(text=query, text_pair=title, max_seq_len=max_seq_length)

    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]

    if not is_test:
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    # 在预测或者评估阶段，不返回 label 字段
    else:
        return input_ids, token_type_ids

In [6]:
### 对训练集的第 1 条数据进行转换
input_ids, token_type_ids, label = convert_example(train_ds[0], tokenizer)

In [7]:
# 为了后续方便使用，我们使用python偏函数（partial）给 convert_example 赋予一些默认参数
from functools import partial

# 训练集和验证集的样本转换函数
trans_func = partial(convert_example, tokenizer=tokenizer, max_seq_length=MAX_SEQ_LENGTH)

#### 组装 Batch 数据 & Padding

上一小节，我们完成了对单条样本的转换，本节我们需要将样本组合成 Batch 数据，对于不等长的数据还需要进行 Padding 操作，便于 GPU 训练。

PaddleNLP 提供了许多关于 NLP 任务中构建有效的数据 pipeline 的常用 API

| API                             | 简介                                       |
| ------------------------------- | :----------------------------------------- |
| `paddlenlp.data.Stack`          | 堆叠N个具有相同shape的输入数据来构建一个batch |
| `paddlenlp.data.Pad`            | 将长度不同的多个句子padding到统一长度，取N个输入数据中的最大长度 |
| `paddlenlp.data.Tuple`          | 将多个batchify函数包装在一起 |

更多数据处理操作详见： [https://paddlenlp.readthedocs.io/zh/latest/data_prepare/data_preprocess.html](https://paddlenlp.readthedocs.io/zh/latest/data_prepare/data_preprocess.html)

In [8]:
from paddlenlp.data import Stack, Pad, Tuple
a = [1, 2, 3, 4]
b = [3, 4, 5, 6]
c = [5, 6, 7, 8]
result = Stack()([a, b, c])
# print("Stacked Data: \n", result)
# print()

a = [1, 2, 3, 4]
b = [5, 6, 7]
c = [8, 9]
result = Pad(pad_val=0)([a, b, c])
# print("Padded Data: \n", result)
# print()

data = [
        [[1, 2, 3, 4], [1]],
        [[5, 6, 7], [0]],
        [[8, 9], [1]],
       ]
batchify_fn = Tuple(Pad(pad_val=0), Stack())
ids, labels = batchify_fn(data)
# print("ids: \n", ids)
# print()
# print("labels: \n", labels)
# print()

In [9]:
# 我们的训练数据会返回 input_ids, token_type_ids, labels 3 个字段
# 因此针对这 3 个字段需要分别定义 3 个组 batch 操作
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]

#### 定义 Dataloader
下面我们基于组 batchify_fn 函数和样本转换函数 trans_func 来构造训练集的 DataLoader, 支持多卡训练


In [10]:
# 定义分布式 Sampler: 自动对训练数据进行切分，支持多卡并行训练
batch_sampler = paddle.io.DistributedBatchSampler(train_ds, batch_size=BATCH_SIZE, shuffle=True)

# 基于 train_ds 定义 train_data_loader
# 因为我们使用了分布式的 DistributedBatchSampler, train_data_loader 会自动对训练数据进行切分
train_data_loader = paddle.io.DataLoader(
        dataset=train_ds.map(trans_func),
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)


# 针对验证集数据加载，我们使用单卡进行评估，所以采用 paddle.io.BatchSampler 即可
# 定义 dev_data_loader
if EVAL:
        batch_sampler = paddle.io.BatchSampler(dev_ds, batch_size=BATCH_SIZE, shuffle=False)
        dev_data_loader = paddle.io.DataLoader(
                dataset=dev_ds.map(trans_func),
                batch_sampler=batch_sampler,
                collate_fn=batchify_fn,
                return_list=True)

### 2.3 模型搭建

自从 2018 年 10 月以来，NLP 个领域的任务都通过 Pretrain + Finetune 的模式相比传统 DNN 方法在效果上取得了显著的提升，本节我们以百度开源的预训练模型 ERNIE-Gram 为基础模型，在此之上构建 Point-wise 语义匹配网络。

首先我们来定义网络结构:

In [11]:
class PointwiseMatching(nn.Layer):
    # 此处的 pretained_model 在本例中会被 ERNIE-Gram 预训练模型初始化
    def __init__(self, pretrained_model, dropout=None):
        super().__init__()
        self.ptm = pretrained_model
        self.dropout = nn.Dropout(dropout if dropout is not None else 0.1)

        # 语义匹配任务: 相似、不相似 2 分类任务
        self.classifier = nn.Linear(self.ptm.config["hidden_size"], 2)

    def forward(self,
                input_ids,
                token_type_ids=None,
                position_ids=None,
                attention_mask=None):

        # 此处的 Input_ids 由两条文本的 token ids 拼接而成
        # token_type_ids 表示两段文本的类型编码
        # 返回的 cls_embedding 就表示这两段文本经过模型的计算之后而得到的语义表示向量
        _, cls_embedding = self.ptm(input_ids, token_type_ids, position_ids, attention_mask)
        cls_embedding = self.dropout(cls_embedding)

        # 基于文本对的语义表示向量进行 2 分类任务
        logits = self.classifier(cls_embedding)
        probs = F.softmax(logits)

        return probs

# 定义 Point-wise 语义匹配网络
model = PointwiseMatching(pretrained_model)

### 2.4 模型训练 & 评估

In [12]:
from paddlenlp.transformers import LinearDecayWithWarmup

num_training_steps = len(train_data_loader) * EPOCHS

# 定义 learning_rate_scheduler，负责在训练过程中对 lr 进行调度
lr_scheduler = LinearDecayWithWarmup(LR, num_training_steps, 0.0)

# Generate parameter names needed to perform weight decay.
# All bias and LayerNorm parameters are excluded.
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义 Optimizer
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=0.0,
    apply_decay_param_fun=lambda x: x in decay_params)

# 采用交叉熵 损失函数
criterion = paddle.nn.loss.CrossEntropyLoss()

# 评估的时候采用准确率指标
metric = paddle.metric.Accuracy()

In [13]:
# 因为训练过程中同时要在验证集进行模型评估，因此我们先定义评估函数

@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader, phase="dev"):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch
        probs = model(input_ids=input_ids, token_type_ids=token_type_ids)
        loss = criterion(probs, labels)
        losses.append(loss.numpy())
        correct = metric.compute(probs, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval {} loss: {:.5}, accu: {:.5}".format(phase, np.mean(losses), accu))
    model.train()
    metric.reset()

In [14]:
%%time

# 接下来，开始正式训练模型，训练时间较长，可注释掉这部分
global_step = 0
tic_train = time.time()

for epoch in range(1, EPOCHS + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        # print(step)
        input_ids, token_type_ids, labels = batch
        probs = model(input_ids=input_ids, token_type_ids=token_type_ids)
        loss = criterion(probs, labels)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        global_step += 1
        
        # 每间隔 10 step 输出训练指标
        if global_step % 10 == 0:
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc, 10 / (time.time() - tic_train)))
            tic_train = time.time()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()

        # 每间隔 100 step 在验证集和测试集上进行评估
        if EVAL:
            if global_step % EVAL_STEP == 0:
                evaluate(model, criterion, metric, dev_data_loader, "dev")
            
# 训练结束后，存储模型参数
save_dir = os.path.join("checkpoint", "model_%d" % global_step)
os.makedirs(save_dir)

save_param_path = os.path.join(save_dir, 'model_state.pdparams')
paddle.save(model.state_dict(), save_param_path)
tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 0.67182, accu: 0.52250, speed: 0.81 step/s
global step 20, epoch: 1, batch: 20, loss: 0.69528, accu: 0.55000, speed: 0.89 step/s
global step 30, epoch: 1, batch: 30, loss: 0.57852, accu: 0.60000, speed: 0.89 step/s
global step 40, epoch: 1, batch: 40, loss: 0.49860, accu: 0.64625, speed: 0.88 step/s
global step 50, epoch: 1, batch: 50, loss: 0.46105, accu: 0.68050, speed: 0.88 step/s
global step 60, epoch: 1, batch: 60, loss: 0.47143, accu: 0.71167, speed: 0.88 step/s
global step 70, epoch: 1, batch: 70, loss: 0.39744, accu: 0.73857, speed: 0.88 step/s
global step 80, epoch: 1, batch: 80, loss: 0.38737, accu: 0.75750, speed: 0.87 step/s
global step 90, epoch: 1, batch: 90, loss: 0.36538, accu: 0.77389, speed: 0.87 step/s
global step 100, epoch: 1, batch: 100, loss: 0.46263, accu: 0.78850, speed: 0.87 step/s
global step 110, epoch: 1, batch: 110, loss: 0.37279, accu: 0.80114, speed: 0.87 step/s
global step 120, epoch: 1, batch: 120, loss: 0.419

基于默认参数配置进行单卡训练大概要持续 4 个小时左右，会训练完成 3 个 Epoch, 模型最终的收敛指标结果如下:


| 数据集 | Accuracy |
| -------- | -------- |
| dev.tsv     | 89.62  |

可以看到: 我们基于 PaddleNLP ，利用 ERNIE-Gram 预训练模型使用非常简洁的代码，就在权威语义匹配数据集上取得了很不错的效果.

### 2.5 模型预测

接下来我们使用已经训练好的语义匹配模型对一些预测数据进行预测。待预测数据为每行都是文本对的 tsv 文件，我们使用 Lcqmc 数据集的测试集作为我们的预测数据，进行预测并提交预测结果到 [千言文本相似度竞赛](https://aistudio.baidu.com/aistudio/competition/detail/45)

下载我们已经训练好的语义匹配模型, 并解压

#### 定义预测函数

In [15]:
def predict(model, data_loader):
    batch_probs = []

    # 预测阶段打开 eval 模式，模型中的 dropout 等操作会关掉
    model.eval()
    n_len = len(data_loader)
    with paddle.no_grad():
        for idx, batch_data in enumerate(data_loader):
            input_ids, token_type_ids = batch_data
            input_ids = paddle.to_tensor(input_ids)
            token_type_ids = paddle.to_tensor(token_type_ids)
            
            # 获取每个样本的预测概率: [BATCH_SIZE, 2] 的矩阵
            batch_prob = model(input_ids=input_ids, token_type_ids=token_type_ids).numpy()
            batch_probs.append(batch_prob)
            print(f'\r{idx+1}/{n_len}', end='')
            
        batch_probs = np.concatenate(batch_probs, axis=0)
        return batch_probs

#### 定义预测数据的 data_loader

In [16]:
# 预测数据的转换函数
# predict 数据没有 label, 因此 convert_exmaple 的 is_test 参数设为 True
trans_func = partial(convert_example, tokenizer=tokenizer, max_seq_length=MAX_SEQ_LENGTH, is_test=True)

# 预测数据的组 batch 操作
# predict 数据只返回 input_ids 和 token_type_ids，因此只需要 2 个 Pad 对象作为 batchify_fn
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment_ids
): [data for data in fn(samples)]

# 加载预测数据
# test_ds = load_dataset("lcqmc", splits=["test"])

In [17]:
def read(data_path):
    for idx, row in data_path.iterrows():
        yield {'query': row.Test_text, 'title': row.Reference_text, 'label': ''}


# data_path为read()方法的参数
test_df = pd.read_csv(f'data/data117130/{TEST_CSV}')
test_ds = load_dataset(read, data_path=test_df, lazy=False)

len(test_ds)

176820

In [18]:
batch_sampler = paddle.io.BatchSampler(test_ds, batch_size=512, shuffle=False) # batch_size=32

# 生成预测数据 data_loader
predict_data_loader = paddle.io.DataLoader(
        dataset=test_ds.map(trans_func),
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)

#### 开始预测

In [19]:
%%time

# 执行预测函数
y_probs = predict(model, predict_data_loader)

346/346CPU times: user 39min 28s, sys: 4min 35s, total: 44min 3s
Wall time: 43min 43s


#### 输出预测结果

In [20]:
# Threshold = 0
# Threshold = 0.5

y_probs_new = y_probs.copy()
# for idx, i in enumerate(y_probs_new[:, 1]):
#     if i > Threshold:
#         pass
#     else:
#         y_probs_new[idx, 1] = 0

# 根据预测概率获取预测 label
y_probs_new = np.argmax(y_probs_new, axis=1)

df_submission = pd.DataFrame({"Test": test_df.Test.values,
                              "Reference": test_df.Reference.values,
#                               "Test_text": test_df.Test_text.values,
#                               "Reference_text": test_df.Reference_text.values,
                              "sentiment": y_probs_new})

df_submission = df_submission[df_submission.sentiment == 1]
df_submission = df_submission.drop(['sentiment'], axis=1)
df_submission.to_csv("submission.csv", index=False)

print(len(df_submission))
df_submission.head()

2098


,Test,Reference
295,1053,52
708,1084,1014
720,1084,345
825,1084,440
866,538,1091
